In [1]:
import numpy as np
import pandas as pd
import pickle 
import torch
from model import BERT4Rec

In [2]:
if torch.cuda.is_available() :
    cuda_aval = True
device = torch.device("cuda" if cuda_aval else "cpu")

In [14]:
with open("./data/input_data.pickle", "rb") as fr :
    raw_data = pickle.load(fr)

unique_sid = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
item_id = unique_sid.to_numpy().reshape(6807)

uim = pd.read_csv("/opt/ml/input/workspace/CF/DL CF/Mult-VAE/new_user_movie_interaction.csv")
user_id = uim["user"].to_numpy()

In [4]:
model = torch.load("./exp/model.pt").to(device)
#model = BERT4Rec()
#model.load_state_dict(torch.load("./exp/model.pth", map_location = device))

In [13]:
max_len = 50

for key in raw_data.keys() :
    
    length = len(raw_data[key])
    if length < max_len : 
        dif = max_len-length
        data = [0]*dif + raw_data[key][length:]
    else :
        data = raw_data[key][-50:]

    data = torch.LongTensor(data).unsqueeze(dim=0)
    
    print(data.shape)
    result = model(data)[0]

    for t in result :
        t = t.detach().cpu()
        top_k_idx = np.argpartition(t, -10)[-10:]
        rec_item_id = 
    break



torch.Size([1, 50])
tensor([2.8560, 3.3233, 2.9957, 3.7041, 3.0162, 3.7562, 2.9912, 3.2045, 3.1161,
        3.0155])
tensor([3.2777, 3.3244, 3.3352, 3.6634, 3.4780, 3.4630, 3.3784, 3.9327, 4.1013,
        3.7808])
tensor([2.4047, 2.4800, 2.4828, 2.9419, 2.6178, 2.5166, 3.0248, 2.5822, 2.4958,
        2.6855])
tensor([2.4287, 2.4930, 2.6956, 2.5809, 2.5589, 2.8725, 2.6212, 2.6461, 2.5788,
        2.6748])
tensor([2.1154, 2.1332, 2.1531, 2.1874, 2.7588, 2.2758, 2.1967, 2.9331, 2.1932,
        2.1950])
tensor([2.8312, 3.0834, 3.0886, 2.9096, 2.8709, 2.9264, 3.3185, 3.4105, 2.8443,
        3.7171])
tensor([2.6681, 2.6828, 2.7404, 2.9525, 3.0167, 2.9602, 2.7744, 2.9807, 3.1570,
        3.2393])
tensor([2.7514, 2.7651, 3.8789, 2.8098, 2.9967, 3.1100, 3.3535, 3.3455, 2.9398,
        3.1033])
tensor([2.8784, 2.9020, 2.9356, 3.0958, 3.1377, 3.0259, 3.1428, 3.8966, 3.4218,
        3.4420])
tensor([3.4871, 3.5208, 3.7749, 3.8410, 3.7346, 3.8836, 4.4493, 4.1017, 3.9471,
        4.4681])
tensor([2.

In [6]:
[0] * 10 + [1,2,3]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3]